In [ ]:
# Import các thư viện cần thiết
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score



In [ ]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv('duong_dan_den_file.csv')

In [ ]:
# Hiển thị một số dòng đầu tiên của dữ liệu để kiểm tra
print(data.head())

In [ ]:
# Chia dữ liệu thành features và target
X = data.drop('target_column_name', axis=1)  # Xác định các features (các cột ngoại trừ cột nhãn)
y = data['target_column_name']  # Nhãn cần dự đoán

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Khởi tạo mô hình KNN với số lân cận là 3
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Huấn luyện mô hình trên tập huấn luyện
knn.fit(X_train, y_train)

In [ ]:
# Dự đoán nhãn của dữ liệu kiểm tra
y_pred = knn.predict(X_test)

# Đánh giá độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)